# Clustering Crypto

In [1]:
# Initial imports
import pandas as pd
import hvplot.pandas
import plotly.express as px
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans

ImportError: cannot import name 'Rectangles' from 'holoviews.element' (/opt/anaconda3/lib/python3.7/site-packages/holoviews/element/__init__.py)

### Deliverable 1: Preprocessing the Data for PCA

In [270]:
# Load the crypto_data.csv dataset.
crypto_df = pd.read_csv("crypto_data.csv")
# print(crypto_df['Algorithm'].value_counts())
# print(f"SIZE: {crypto_df.count()}")
print(crypto_df['IsTrading'].value_counts())

True     1144
False     108
Name: IsTrading, dtype: int64


In [271]:
# Keep only the cryptocurrencies that are being traded.
crypto_df = crypto_df.loc[crypto_df['IsTrading']==True]
print(crypto_df['IsTrading'].value_counts())

True    1144
Name: IsTrading, dtype: int64


In [272]:
# Keep only the cryptocurrencies that have a working algorithm.
pd.isna(crypto_df['Algorithm'])

0       False
1       False
2       False
3       False
4       False
        ...  
1243    False
1244    False
1245    False
1246    False
1247    False
Name: Algorithm, Length: 1144, dtype: bool

In [273]:
# Remove the "IsTrading" column. 
crypto_df=crypto_df.drop(columns=['IsTrading'])
crypto_df.head()

,Unnamed: 0,CoinName,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
0,42,42 Coin,Scrypt,PoW/PoS,4.199995e+01,42
1,365,365Coin,X11,PoW/PoS,NaN,2300000000
2,404,404Coin,Scrypt,PoW/PoS,1.055185e+09,532000000
3,611,SixEleven,SHA-256,PoW,NaN,611000
4,808,808,SHA-256,PoW/PoS,0.000000e+00,0


In [274]:
# Remove rows that have at least 1 null value.
print(f"Total Rows: {len(crypto_df.index)}")
print(f"Null Rows: {crypto_df['TotalCoinsMined'].isnull().sum()}")
print(f"Non-null Rows: {len(crypto_df.index)-crypto_df['TotalCoinsMined'].isnull().sum()}")

crypto_df=crypto_df.dropna()
print(f"New Size: {len(crypto_df.index)}")

Total Rows: 1144
Null Rows: 459
Non-null Rows: 685
New Size: 685


In [275]:
# Keep the rows where coins are mined.
print(f"Coins Mined: {crypto_df.loc[crypto_df['TotalCoinsMined']>0].shape[0]}")
print(f"Coins Unmined: {crypto_df.loc[crypto_df['TotalCoinsMined']<=0].shape[0]}")
crypto_df=crypto_df.loc[crypto_df['TotalCoinsMined']>0]
print(f"New DF Size: {crypto_df.shape[0]}")

Coins Mined: 532
Coins Unmined: 153
New DF Size: 532


In [276]:
# Create a new DataFrame that holds only the cryptocurrencies names. 
cnames_df = crypto_df.loc[:, crypto_df.columns.intersection(['Unnamed: 0','CoinName'])].set_index('Unnamed: 0')
cnames_df.index.name = None
cnames_df

,CoinName
42,42 Coin
404,404Coin
1337,EliteCoin
BTC,Bitcoin
ETH,Ethereum
...,...
ZEPH,ZEPHYR
GAP,Gapcoin
BDX,Beldex
ZEN,Horizen


In [277]:
# Drop the 'CoinName' column since it's not going to be used on the clustering algorithm.
crypto_df=crypto_df.drop(columns='CoinName').set_index('Unnamed: 0')
crypto_df.index.name = None
crypto_df

,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
42,Scrypt,PoW/PoS,4.199995e+01,42
404,Scrypt,PoW/PoS,1.055185e+09,532000000
1337,X13,PoW/PoS,2.927942e+10,314159265359
BTC,SHA-256,PoW,1.792718e+07,21000000
ETH,Ethash,PoW,1.076842e+08,0
...,...,...,...,...
ZEPH,SHA-256,DPoS,2.000000e+09,2000000000
GAP,Scrypt,PoW/PoS,1.493105e+07,250000000
BDX,CryptoNight,PoW,9.802226e+08,1400222610
ZEN,Equihash,PoW,7.296538e+06,21000000


In [278]:
# Use get_dummies() to create variables for text features.
X = pd.get_dummies(data=crypto_df, columns=['Algorithm', 'ProofType'])
X 

,TotalCoinsMined,TotalCoinSupply,Algorithm_1GB AES Pattern Search,Algorithm_536,Algorithm_Argon2d,Algorithm_BLAKE256,Algorithm_Blake,Algorithm_Blake2S,Algorithm_Blake2b,Algorithm_C11,...,ProofType_PoW/PoS,ProofType_PoW/PoS,ProofType_PoW/PoW,ProofType_PoW/nPoS,ProofType_Pos,ProofType_Proof of Authority,ProofType_Proof of Trust,ProofType_TPoS,ProofType_Zero-Knowledge Proof,ProofType_dPoW/PoW
42,4.199995e+01,42,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
404,1.055185e+09,532000000,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
1337,2.927942e+10,314159265359,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
BTC,1.792718e+07,21000000,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
ETH,1.076842e+08,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ZEPH,2.000000e+09,2000000000,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
GAP,1.493105e+07,250000000,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
BDX,9.802226e+08,1400222610,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
ZEN,7.296538e+06,21000000,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [279]:
# Standardize the data with StandardScaler().
X_scaled = StandardScaler().fit_transform(X)
print(X_scaled[0:2])

[[-0.11710817 -0.1528703  -0.0433963  -0.0433963  -0.0433963  -0.06142951
  -0.07530656 -0.0433963  -0.06142951 -0.06142951 -0.0433963  -0.0433963
  -0.19245009 -0.06142951 -0.09740465 -0.0433963  -0.11547005 -0.07530656
  -0.0433963  -0.0433963  -0.15191091 -0.0433963  -0.13118084 -0.0433963
  -0.0433963  -0.08703883 -0.0433963  -0.0433963  -0.0433963  -0.0433963
  -0.06142951 -0.0433963  -0.08703883 -0.08703883 -0.08703883 -0.0433963
  -0.13118084 -0.13840913 -0.13840913 -0.0433963  -0.06142951 -0.0433963
  -0.07530656 -0.18168574 -0.0433963  -0.0433963  -0.0433963  -0.07530656
  -0.15826614 -0.31491833 -0.0433963  -0.08703883 -0.07530656 -0.06142951
   1.38675049 -0.0433963  -0.0433963  -0.06142951 -0.0433963  -0.0433963
  -0.0433963  -0.0433963  -0.0433963  -0.0433963  -0.0433963  -0.0433963
  -0.39879994 -0.0433963  -0.18168574 -0.0433963  -0.08703883 -0.08703883
  -0.10680283 -0.0433963  -0.13118084 -0.0433963  -0.0433963  -0.0433963
  -0.0433963  -0.07530656 -0.43911856 -0.04339

### Deliverable 2: Reducing Data Dimensions Using PCA

In [280]:
# Using PCA to reduce dimension to three principal components.
pca = PCA(n_components=3)
crypto_pca = pca.fit_transform(X_scaled)
crypto_pca

array([[-0.34152838,  1.01304331, -0.58879869],
       [-0.32482341,  1.01348205, -0.58929166],
       [ 2.31941602,  1.7725028 , -0.71427891],
       ...,
       [ 0.33521319, -2.23948072,  0.42836969],
       [-0.15008708, -2.07764674,  0.46144078],
       [-0.30089938,  0.79257984, -0.27402043]])

In [281]:
# Create a DataFrame with the three principal components.
pcs_df = pd.DataFrame(crypto_pca, columns=["PC 1", "PC 2", "PC 3"], index=X.index)
pcs_df.head(10)

,PC 1,PC 2,PC 3
42,-0.341528,1.013043,-0.588799
404,-0.324823,1.013482,-0.589292
1337,2.319416,1.772503,-0.714279
BTC,-0.138409,-1.331075,0.185172
ETH,-0.145918,-2.013787,0.405640
LTC,-0.172631,-1.145640,-0.000357
DASH,-0.382723,1.285212,-0.521535
XMR,-0.150387,-2.218789,0.365336
ETC,-0.144355,-2.013853,0.405614
ZEC,-0.150086,-2.077647,0.461441


### Deliverable 3: Clustering Crytocurrencies Using K-Means

#### Finding the Best Value for `k` Using the Elbow Curve

In [282]:
# Create an elbow curve to find the best value for K.
inertia = []
K = list(range(1, 11))

for i in K:
   km = KMeans(n_clusters=i, random_state=0)
   km.fit(pcs_df)
   inertia.append(km.inertia_)

elbow_data = {"k": K, "inertia": inertia}
elbow_df = pd.DataFrame(elbow_data)
# elbow_df.hvplot.line(x="k", y="inertia", title="Elbow Curve", xticks=K)

Running K-Means with `k=4`

In [283]:
# Initialize the K-Means model.
model = KMeans(n_clusters=3, random_state=5)
model

# Fit the model
model.fit(pcs_df)

# Predict clusters
predictions = model.predict(pcs_df)
print(predictions)

[2 2 2 0 0 0 2 0 0 0 2 0 2 2 0 2 0 0 2 2 0 0 0 0 0 2 0 0 0 2 0 2 0 0 2 2 0
 0 0 0 0 0 2 2 0 0 0 0 0 2 2 0 2 0 0 0 0 2 0 0 2 0 2 2 2 0 0 0 2 2 2 2 2 0
 0 0 2 2 0 2 0 2 2 0 0 0 0 2 2 0 2 0 0 2 2 0 2 2 0 0 2 2 0 2 2 0 2 0 2 0 2
 0 2 2 0 0 2 0 0 0 2 0 0 0 0 0 2 2 0 0 0 2 0 2 0 0 2 0 2 0 2 2 0 0 2 0 0 2
 2 0 2 0 2 2 2 0 0 0 0 2 2 2 2 2 0 0 2 2 2 2 2 0 2 2 2 2 2 0 2 0 2 2 0 2 0
 2 2 0 2 0 2 0 2 0 2 2 2 2 0 2 2 2 2 2 0 0 2 2 0 0 2 2 2 2 2 0 2 2 2 2 2 2
 2 2 0 2 2 2 2 2 2 0 0 0 2 2 2 2 0 2 0 2 2 0 2 0 0 2 0 0 2 0 2 2 2 0 2 2 0
 2 2 2 2 2 2 2 0 2 0 2 2 2 2 0 2 0 2 0 0 0 0 2 0 2 2 0 2 0 0 0 2 0 2 0 0 0
 2 0 2 0 2 2 2 0 2 0 0 0 0 0 2 2 0 2 2 2 0 2 0 2 0 2 0 2 2 2 2 0 2 2 0 2 2
 2 0 0 0 0 2 2 2 2 0 2 0 0 0 2 2 0 0 2 2 0 2 0 0 0 2 0 0 2 2 2 0 0 0 2 2 2
 0 0 2 0 0 0 0 2 2 2 0 0 0 2 2 2 2 2 2 0 0 0 0 2 2 2 0 2 0 2 2 2 2 0 2 2 0
 2 2 0 0 2 0 2 0 0 0 0 2 2 0 2 0 2 2 2 2 2 2 0 0 0 2 2 2 2 2 2 0 2 0 0 0 0
 2 2 2 2 0 2 2 0 2 2 0 2 0 2 0 0 2 2 0 2 0 0 0 0 0 2 0 2 0 2 2 0 2 2 2 2 2
 0 0 0 2 2 2 0 2 0 2 0 2 

In [284]:
#  Add a new column, "Class" to the clustered_df DataFrame that holds the predictions.
pcs_df["class"] = model.labels_
pcs_df.head()
# Create a new DataFrame including predicted clusters and cryptocurrencies features.
# Concatentate the crypto_df and pcs_df DataFrames on the same columns.
clustered_df = pd.concat([crypto_df,pcs_df],axis=1)
clustered_df

#  Add a new column, "CoinName" to the clustered_df DataFrame that holds the names of the cryptocurrencies. 
clustered_df = pd.concat([clustered_df,cnames_df],axis=1)


# Print the shape of the clustered_df
print(clustered_df.shape)
clustered_df.head(10)

(532, 9)


,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply,PC 1,PC 2,PC 3,class,CoinName
42,Scrypt,PoW/PoS,4.199995e+01,42,-0.341528,1.013043,-0.588799,2,42 Coin
404,Scrypt,PoW/PoS,1.055185e+09,532000000,-0.324823,1.013482,-0.589292,2,404Coin
1337,X13,PoW/PoS,2.927942e+10,314159265359,2.319416,1.772503,-0.714279,2,EliteCoin
BTC,SHA-256,PoW,1.792718e+07,21000000,-0.138409,-1.331075,0.185172,0,Bitcoin
ETH,Ethash,PoW,1.076842e+08,0,-0.145918,-2.013787,0.405640,0,Ethereum
LTC,Scrypt,PoW,6.303924e+07,84000000,-0.172631,-1.145640,-0.000357,0,Litecoin
DASH,X11,PoW/PoS,9.031294e+06,22000000,-0.382723,1.285212,-0.521535,2,Dash
XMR,CryptoNight-V7,PoW,1.720114e+07,0,-0.150387,-2.218789,0.365336,0,Monero
ETC,Ethash,PoW,1.133597e+08,210000000,-0.144355,-2.013853,0.405614,0,Ethereum Classic
ZEC,Equihash,PoW,7.383056e+06,21000000,-0.150086,-2.077647,0.461441,0,ZCash


### Deliverable 4: Visualizing Cryptocurrencies Results

#### 3D-Scatter with Clusters

In [285]:
# Creating a 3D-Scatter with the PCA data and the clusters
fig = px.scatter_3d(
    clustered_df,
    x="PC1",
    y="PC2",
    z="PC3",
    color="class",
    symbol="class",
    width=800,
)
fig.update_layout(legend=dict(x=0, y=1))
fig.show()

In [286]:
# Create a table with tradable cryptocurrencies.
clustered_df.hvplot.table(sortable=True, selectable=True)

In [287]:
# Print the total number of tradable cryptocurrencies.
clustered_df.shape[0]

532

In [288]:
# Scaling data to create the scatter plot with tradable cryptocurrencies.
scaled = MinMaxScaler().fit_transform(clustered_df[['TotalCoinsMined','TotalCoinSupply']])
scaled

array([[0.00000000e+00, 4.20000000e-11],
       [1.06585544e-03, 5.32000000e-04],
       [2.95755135e-02, 3.14159265e-01],
       ...,
       [9.90135079e-04, 1.40022261e-03],
       [7.37028150e-06, 2.10000000e-05],
       [1.29582282e-07, 1.00000000e-06]])

In [294]:
# Create a new DataFrame that has the scaled data with the clustered_df DataFrame index.
plot_df = pd.DataFrame(data=scaled, columns=["TotalCoinsMined","TotalCoinSupply"], index=clustered_df.index)

# Add the "CoinName" column from the clustered_df DataFrame to the new DataFrame.
# Add the "Class" column from the clustered_df DataFrame to the new DataFrame. 
plot_df = pd.concat([plot_df,clustered_df[["CoinName","class"]]], axis=1)

plot_df.head(10)

,TotalCoinsMined,TotalCoinSupply,CoinName,class
42,0.000000,4.200000e-11,42 Coin,2
404,0.001066,5.320000e-04,404Coin,2
1337,0.029576,3.141593e-01,EliteCoin,2
BTC,0.000018,2.100000e-05,Bitcoin,0
ETH,0.000109,0.000000e+00,Ethereum,0
LTC,0.000064,8.400000e-05,Litecoin,0
DASH,0.000009,2.200000e-05,Dash,2
XMR,0.000017,0.000000e+00,Monero,0
ETC,0.000115,2.100000e-04,Ethereum Classic,0
ZEC,0.000007,2.100000e-05,ZCash,0


In [290]:
# Create a hvplot.scatter plot using x="TotalCoinsMined" and y="TotalCoinSupply".

